In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Snhu1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Image and unique identifier with interactive filtering options, datatable and a chart with geolocation side by side
app.layout = html.Div([
    # Grazioso Salvare Logo (centered)
    html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'width':'300px', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}
    ),
    # Dashboard Title and Unique Identifier
    html.Center(html.B(html.H1('CS 340 - Dashboard- Asia Mayfield'))),
    html.Hr(),
        
    # Interactive filtering options
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'All', 'value': 'all'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Wilderness Rescue', 'value': 'wilderness'},
            {'label': 'Disaster/Tracking', 'value': 'disaster'},
        ],
        value='all',  # Defaults to showing all records
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    ),

    html.Hr(),
    
    # Interactive DataTable
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],       # highlights the first row by default
        page_action="native",
        page_current=0,
        page_size=10,            # 10 rows per page for user friendliness
        style_table={'overflowX': 'auto'}
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that the chart and the geolocation chart are side by side
    html.Div(className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
# Updates the data table based on the selected rescue type filter. Uses MongoDB queries from the Dashboard Specifications Document.
    if filter_type == 'all':
        result = db.read({})
    elif filter_type == 'water':
        result = db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif filter_type == 'wilderness':
        result = db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        })
    elif filter_type == 'disaster':
        result = db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        })
    else:
        result = db.read({})
    df_filtered = pd.DataFrame.from_records(result)
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Displays the breeds of animal based on quantity represented in the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    
    # Updates the graph widget (pie chart) to show breed distribution of the currently filtered data table.
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []
    return [
        dcc.Graph(
            figure = px.pie(dff, names='breed', title='Breed Distribution')
        )
    ]

#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:  # Handles none or empty list
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    """
    Updates the map widget to show the geolocation of the selected animal.
    Uses the selected row from the DataTable and plots it on the map.
    """
    if viewData is None or index is None or len(index) == 0:
        return []
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return []
    row = index[0]
    # Safely grabs columns based on name in case order changes
    lat_col = [col for col in dff.columns if 'lat' in col.lower()][0]
    lon_col = [col for col in dff.columns if 'long' in col.lower()][0]
    breed_col = [col for col in dff.columns if 'breed' in col.lower()][0]
    name_col = [col for col in dff.columns if 'name' in col.lower()][0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(
                position=[float(dff.loc[row, lat_col]), float(dff.loc[row, lon_col])],
                children=[
                    dl.Tooltip(str(dff.loc[row, breed_col])),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(str(dff.loc[row, name_col]))
                    ])
                ]
            )
        ])
    ]

app.run_server(debug=True)


Dash app running on http://127.0.0.1:9787/
